# Some tests with Symmetric Cryptography

In [ ]:
# Some imports
from os import urandom
from binascii import hexlify
import time
import collections
import numpy as np
# import matplotlib.pyplot as plt
import requests

In [ ]:
%pip install -q mediapy
import mediapy as media

In [ ]:
#if you get an error about of missing package, uncomment the following line
%pip install -q cryptography 

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.exceptions import InvalidTag
# from cryptography.hazmat.backends import default_backend

In [ ]:
# We use AES-128
# key size is 16 bytes
KEYLEN = 16
# block size is 16 bytes
BLOCKLEN = 16

## Example 1
We will encrypt a single block of data using AES in ECB mode

In [ ]:
# set plaintext block p to the all-zero string
p = b'\x00'*BLOCKLEN
# print as hex string
print("p = {}".format(hexlify(p)))

In [ ]:
# pick a random key using Python's crypto PRNG
k = urandom(KEYLEN)
# print as hex string
print("k = {}".format(hexlify(k)))

In [ ]:
# create an instance of AES-128 to encrypt a single block
cipher = Cipher(algorithms.AES(k), modes.ECB())

In [ ]:
# encrypt plaintext p to ciphertext c
aes_encrypt = cipher.encryptor()
c = aes_encrypt.update(p) + aes_encrypt.finalize()
print ("E({},{}) = {}".format(hexlify(k),hexlify(p), hexlify(c)))

In [ ]:
# decrypt ciphertext c to plaintext p
aes_decrypt = cipher.decryptor()
p = aes_decrypt.update(c) + aes_decrypt.finalize()
print("D({},{}) = {}".format(hexlify(k),hexlify(c), hexlify(p)))

## Example 2
We encrypt two identical blocks with ECB and see that their encryption is the same

In [ ]:
# a function that pretty prints the plaintext into blocks
def blocks(data):
    split = [hexlify(data[i:i+BLOCKLEN]) for i in range(0, len(data), BLOCKLEN)]
    return b' '.join(split)

In [ ]:
# set plaintext block p to the all-zero string
p = b'\x00'*BLOCKLEN*2
print("p = {}".format(blocks(p)))

In [ ]:
# pick a random key using Python's crypto PRNG
k = urandom(KEYLEN)
print("k = {}".format(hexlify(k)))

In [ ]:
# create an instance of AES-128 to encrypt and decrypt
cipher = Cipher(algorithms.AES(k), modes.ECB())

In [ ]:
# encrypt plaintext p to ciphertext c
aes_encrypt = cipher.encryptor()
c = aes_encrypt.update(p) + aes_encrypt.finalize()
print("enc({}) =\n    {}".format(blocks(p), blocks(c)))

## Example 3
We encrypt the Linux mascot with ecb, and see that patterns get through. According to Marsh Ray,
> "ECB cannot be used because you can see the penguin"

In [ ]:
# Load the penguin image
# Uncomment the following line if you get an error about a missing file
#!wget -q https://raw.githubusercontent.com/gverticale/network-security-and-cryptography/master/tux_gray.png
tux_png = media.read_image('https://raw.githubusercontent.com/gverticale/network-security-and-cryptography/master/tux_gray.png')

media.show_image(tux_png,height=480, title='Tux')

In [ ]:
# Keep only luminance and serialize the image
# The plaintext is a string of bytes
tux = tux_png[:,:,1] * 255
tux = tux.astype(np.uint8)
p = tux.tobytes()

In [ ]:
# Generate a key and an instance of AES
k = urandom(KEYLEN)
cipher = Cipher(algorithms.AES(k), modes.ECB())

In [ ]:
# Encrypt the image
aes_encrypt = cipher.encryptor()
c = aes_encrypt.update(p) + aes_encrypt.finalize()

In [ ]:
# Reshape the ciphertext into a matrix of bytes 
# Show the ciphertext
encrypted_tux = np.frombuffer(c, dtype = np.uint8).reshape(tux.shape)

media.show_image(encrypted_tux,height=480, title='Encrypted Tux')

## Example 4
We encrypt the Linux mascot with deterministic CTR, and see that patterns disappear.

In [ ]:
# Generate a key and an instance of AES
k = urandom(KEYLEN)

In [ ]:
# Generate the starting counter (in deterministic CTR, we start from 0)
iv = b'\x00'*BLOCKLEN

In [ ]:
# Generate an instance of AES with the given key and nonce
cipher = Cipher(algorithms.AES(k), modes.CTR(iv))

In [ ]:
# Encrypt the image
aes_encrypt = cipher.encryptor()
c = aes_encrypt.update(p) + aes_encrypt.finalize()

In [ ]:
# Reshape the ciphertext into a matrix of bytes 
# Show the ciphertext
encrypted_tux = np.frombuffer(c, dtype = np.uint8).reshape(tux.shape)

media.show_image(encrypted_tux,height=480, title='Encrypted Tux')

## Example 5
We generate a new ciphertext with the same key and still use deterministic CTR.
We then see that we can then learn about the plaintext 

In [ ]:
# Load "The Black Cat" by Edgar Allan Poe 
# Uncomment the following line if you get an error about a missing file
# !wget -q https://raw.githubusercontent.com/gverticale/network-security-and-cryptography/master/cat.txt

catfile = requests.get('https://raw.githubusercontent.com/gverticale/network-security-and-cryptography/master/cat.txt')
cat = catfile.content

#with open('cat.txt', 'rb') as myfile:
#    cat=myfile.read()

# The file is shorter than the image, so we make it the same size
plaintext_cat = 50 * cat
plaintext_cat = plaintext_cat[0:len(p)]

In [ ]:
# Encrypt the text with the same key and IV
cipher = Cipher(algorithms.AES(k), modes.CTR(iv))
aes_encrypt = cipher.encryptor()
encrypted_cat = aes_encrypt.update(plaintext_cat) + aes_encrypt.finalize()

In [ ]:
# Reshape the encrypted poem into a matrix
encrypted_cat_rect = np.frombuffer(encrypted_cat, dtype = np.uint8).reshape(tux.shape)

In [ ]:
# Calculate the XOR between the ciphertexts
ciphertext_xor = np.bitwise_xor(encrypted_cat_rect, encrypted_tux)

media.show_image(ciphertext_xor,height=480, title='Xored Tux')

# Example 6

Let's use an authenticated cipher: AES-GCM

In [ ]:
# Generate a key and an instance of AES-GCM
# The standard requires an 96-bit random IV
k = urandom(KEYLEN)
iv = urandom(12)
cipher = Cipher(algorithms.AES(k), modes.GCM(iv))

In [ ]:
# Encrypt "The Black Cat" and authenticate the string "A poem"
gcm_encrypt = cipher.encryptor()
gcm_encrypt.authenticate_additional_data(b"A poem")
encrypted_cat = gcm_encrypt.update(plaintext_cat) + gcm_encrypt.finalize()
tag = gcm_encrypt.tag

In [ ]:
# Now decrypt
cipher = Cipher(algorithms.AES(k), modes.GCM(iv,tag))

In [ ]:
gcm_decrypt = cipher.decryptor()
gcm_decrypt.authenticate_additional_data(b"A new poem")
try:
    decrypted_cat = gcm_decrypt.update(encrypted_cat) + gcm_decrypt.finalize()
    print(decrypted_cat)
except InvalidTag:
    print("Not authentic")

# Lab Activity: randomized CTR

1. Encrypt tux with randomized CTR
2. Encrypt cat with randomized CTR
3. Compute the bitwise xor of the ciphertexts
4. Show the result

# Lab Activity: Malleability of AES-CTR

1. Start from "THE BLACK CAT"
2. Encrypt it using AES-CTR (use a random IV)
3. Using ciphertext malleability change the ciphertext so that the title of the poem becomes "THE GREEN CAT"
4. Decrypt it showing the modified text
5. Repeat using AES-GCM and verify that it fails

_Quick note on the python library: the output of the encryption is immutable, you need to convert it to a list to modify it (command `list`) , then you need to convert it back to a bytes object (command `bytes`)_